In [91]:
# read vcf data
import sys
import os
import re
import argparse
import pandas as pd
import numpy as np  
import gzip
import pysam
import io
from struct_lmm import StructLMM

In [92]:
CONVERSION = {"0|0":1, "0|1":0, "1|0":0, "1|1":0, "./.":np.nan}

In [93]:
GRM_P  = "GRM_dense_matrix.csv"

In [94]:
def read_vcf(filename):
    with gzip.open(filename, 'rt') as f:
        lines = [l for l in f if not l.startswith("##")]  # Skip meta-information lines
    # print(lines[0])
    df = pd.read_csv(
        io.StringIO("".join(lines)),  # Use io.StringIO instead of pandas.compat.StringIO
        sep="\t", 
        header= 0, 

    )
    return df

# Read the VCF file into a DataFrame
vcf_df = read_vcf("hlaa.vcf.gz")
# print(vcf_df.head())

In [95]:
vcf_df

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,HG00096,...,NA21128,NA21129,NA21130,NA21133,NA21135,NA21137,NA21141,NA21142,NA21143,NA21144
0,6,29942509,.,C,T,.,PASS,AC=4260;AN=5096;DP=71384;AF=0.84;EAS_AF=0.96;E...,GT,1|1,...,1|1,1|1,1|0,1|1,1|1,1|1,1|1,0|1,1|1,1|1
1,6,29942527,.,G,C,.,PASS,AC=1;AN=5096;DP=96102;AF=0;EAS_AF=0;EUR_AF=0;A...,GT,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
2,6,29942547,.,G,C,.,PASS,AC=592;AN=5096;DP=112023;AF=0.12;EAS_AF=0.3;EU...,GT,0|0,...,0|1,0|0,0|0,0|0,0|0,0|1,0|0,0|0,1|0,0|0
3,6,29942553,.,G,A,.,PASS,AC=49;AN=5096;DP=116511;AF=0.01;EAS_AF=0;EUR_A...,GT,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
4,6,29942559,.,C,G,.,PASS,AC=702;AN=5096;DP=118281;AF=0.14;EAS_AF=0.14;E...,GT,0|0,...,0|0,0|0,0|0,1|0,0|1,0|0,0|0,0|0,0|0,0|1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,6,29946370,.,T,A,.,PASS,AC=19;AN=5096;DP=25177;AF=0;EAS_AF=0;EUR_AF=0;...,GT,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
266,6,29946382,.,A,G,.,PASS,AC=1;AN=5096;DP=25321;AF=0;EAS_AF=0;EUR_AF=0;A...,GT,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
267,6,29946391,.,T,C,.,PASS,AC=19;AN=5096;DP=25202;AF=0;EAS_AF=0;EUR_AF=0;...,GT,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
268,6,29946442,.,T,C,.,PASS,AC=19;AN=5096;DP=25658;AF=0;EAS_AF=0;EUR_AF=0;...,GT,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0


In [96]:
GRM = pd.read_csv(GRM_P, header=0, index_col=0)

In [97]:
vcf_df = vcf_df.loc[:, vcf_df.columns.str.startswith(("NA", "HG"))]
vcf_df = vcf_df.loc[:, GRM.columns]

In [98]:
vcf_df = vcf_df.applymap(lambda x: CONVERSION[x])

In [99]:
y = vcf_df.iloc[1].to_numpy() # taking sample 0 
v = vcf_df.iloc[2].to_numpy() # taking sample 0 

In [100]:
GRM

,HG00759,HG00766,HG00844,HG00851,HG00864,HG00867,HG00879,HG00881,HG00956,HG00978,...,HG02398,HG02399,HG02401,HG02402,HG02405,HG02406,HG02407,HG02408,HG02409,HG02410
HG00759,0.669228,0.071460,-0.020176,0.121861,0.284350,0.114262,0.197725,-0.020176,-0.020176,0.364764,...,0.126667,0.673281,0.536826,0.171456,0.645597,0.550448,0.653501,0.645597,0.525441,0.157571
HG00766,0.071460,0.796105,0.123361,0.166827,0.108031,0.183603,0.166931,0.123361,0.123361,0.214652,...,0.086547,0.029811,0.233876,0.057832,0.020933,0.100957,0.045974,0.020933,0.003162,0.168622
HG00844,-0.020176,0.123361,0.277810,-0.235902,-0.077499,-0.114794,-0.316962,0.402062,0.402062,-0.002005,...,0.245983,-0.028781,-0.049217,0.285050,-0.031914,0.002341,-0.036889,-0.031914,-0.039118,-0.296906
HG00851,0.121861,0.166827,-0.235902,1.988834,0.375417,1.755175,1.940687,-0.235902,-0.235902,0.147097,...,0.077203,0.137477,0.213712,-0.081890,0.160142,0.112693,0.211703,0.160142,0.197235,2.015249
HG00864,0.284350,0.108031,-0.077499,0.375417,0.951746,0.251828,0.345357,-0.077499,-0.077499,0.300834,...,0.125605,0.252154,0.257784,-0.008465,0.272724,0.268199,0.364356,0.272724,0.244368,0.311703
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HG02406,0.550448,0.100957,0.002341,0.112693,0.268199,0.137980,0.173235,0.002341,0.002341,0.404765,...,0.183418,0.557226,0.570276,0.105453,0.571744,0.620128,0.568356,0.571744,0.547062,0.211000
HG02407,0.653501,0.045974,-0.036889,0.211703,0.364356,0.201721,0.294227,-0.036889,-0.036889,0.429711,...,0.132676,0.566926,0.554889,0.182666,0.584925,0.568356,0.631481,0.584925,0.536863,0.257314
HG02408,0.645597,0.020933,-0.031914,0.160142,0.272724,0.151263,0.239600,-0.031914,-0.031914,0.388767,...,0.116791,0.705209,0.558157,0.164024,0.723088,0.571744,0.584925,0.630598,0.543067,0.240649
HG02409,0.525441,0.003162,-0.039118,0.197235,0.244368,0.182927,0.295688,-0.039118,-0.039118,0.371052,...,0.177091,0.530469,0.533891,0.232974,0.543067,0.547062,0.536863,0.543067,0.578151,0.261960


In [101]:
E = GRM.to_numpy()

In [102]:
M = np.ones((100, 1))

In [103]:
y.shape, E.shape, M.shape

((100,), (100, 100), (100, 1))

In [ ]:
%%time
lmm = StructLMM(y, M, E)
lmm.fit()

CPU times: user 561 ms, sys: 402 ms, total: 962 ms
Wall time: 424 ms


In [105]:
lmm._P(v)

array([-3.40000000e-03, -3.40000003e-03,  6.71078606e+04,  6.71078606e+04,
       -3.40000006e-03,  6.71078606e+04,  6.71078606e+04,  6.71078606e+04,
        6.71078606e+04, -3.40000002e-03,  6.71078606e+04,  6.71078606e+04,
        6.71078606e+04, -3.40000007e-03, -3.39999994e-03,  6.71078606e+04,
       -3.40000003e-03, -3.40000004e-03,  6.71078606e+04, -3.39999997e-03,
       -3.39999995e-03, -3.40000004e-03,  6.71078606e+04,  6.71078606e+04,
        6.71078606e+04, -3.39999997e-03, -3.39999998e-03, -3.39999994e-03,
       -3.40000004e-03, -3.40000001e-03,  6.71078606e+04, -3.40000002e-03,
       -3.39999999e-03, -3.40000003e-03, -3.40000000e-03, -3.39999997e-03,
        6.71078606e+04, -3.40000007e-03,  6.71078606e+04, -3.40000010e-03,
       -3.39999998e-03, -3.39999997e-03, -3.39999998e-03, -3.40000006e-03,
       -3.40000005e-03,  6.71078606e+04,  6.71078606e+04, -3.39999999e-03,
        6.71078606e+04,  6.71078606e+04, -3.39999992e-03, -3.40000005e-03,
       -3.40000001e-03,  